In [1]:
# Import packages for ...
## pulling data from XML and HTML files
from bs4 import BeautifulSoup 

## automating web browser interaction
from selenium import webdriver # module containing implementations of browser drivers
from webdriver_manager.chrome import ChromeDriverManager # Chrome driver 
from selenium.webdriver.support import expected_conditions as EC # method for writing code that waits until conditions are met 
from selenium.webdriver.support.ui import WebDriverWait # method for writing code that implements implicit or explicit waits
from selenium.webdriver.common.by import By # method for locating elements by their attributes
from selenium.webdriver import ActionChains # module for implementing browser interactions 

## data manipulation
import pandas as pd
from datetime import datetime
import numpy as np
import time
import os

## plotting
import matplotlib.pyplot as plt

In [2]:
# Open up a chrome page
driver = webdriver.Chrome(ChromeDriverManager().install())
url = "https://ida.ussc.gov/analytics/saw.dll?Dashboard"
driver.get(url)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147


[WDM] - Driver [/Users/kate/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [17]:
# Open a dropdown of a particular category, toggle a dropdown for a known value and update the dashboard
def one_checkbox(checkbox_value, check_status, category, val=1):
    """
    checkbox_value: "White", "Black", "Hispanic", "Other"
    checked_status: "true" or "None"
    category: "Race", "Crime Type"
    val: for "Other" the val is either 1 or 0
    """
    
    num=filters.get(category)
    dropdown = driver.find_elements_by_xpath("//img[@src='/bicustom/res/s_IDA/master/selectdropdown_ena.png']")[num]
    dropdown.click()
    time.sleep(2)
    
    if checkbox_value == "Other":
        if category == "Race":
            parent_elem = driver.find_elements_by_xpath("//div[@title='" + checkbox_value + "']")[val]
        elif category == "Crime Type":
            parent_elem = driver.find_elements_by_xpath("//div[@title='" + checkbox_value + "']")[val]
    else:
        parent_elem = driver.find_element_by_xpath("//div[@title='" + checkbox_value + "']")

    child_elements = parent_elem.find_element_by_xpath(".//*").find_element_by_xpath(".//*")

    if child_elements.get_attribute("type") == 'checkbox':
        print("Element is a checkbox")

        if check_status == child_elements.get_attribute("checked"):
            print("Checkbox status as expected: {}".format(checkbox_value))
        else:
            # Select the checkbox
            child_elements.click()
            # Click out so that the page can reload
            driver.find_element_by_xpath("//body").click()
            print("Checkbox updated: {}".format(checkbox_value)) 

    else:
        print("Element is not a checkbox")
        
    num=filters.get(category)
    dropdown = driver.find_elements_by_xpath("//img[@src='/bicustom/res/s_IDA/master/selectdropdown_ena.png']")[num].click()
    

## Import .py function

In [78]:
from sentencing_outcomes import *

In [77]:
# Reload the script
import sys, importlib
importlib.reload(sys.modules['sentencing_outcomes'])

<module 'sentencing_outcomes' from '/Users/kate/Documents/Data Science/dashboard-web-scraping/sentencing_outcomes.py'>

In [ ]:
# use expand list when gathering data from tables
expand_list()

### Run stuff - Sentence Length

In [7]:
# Navigate to Sentencing Outcomes, Sentence Length page
nav_to_sentencingoutcomes(driver, "Sentence Length")

In [79]:
sentence_length_all = pd.DataFrame() # Instantiate a df
sentence_length_all['Race'] = ""
sentence_length_all['Crime Type'] = ""

In [41]:
#r = race[0] # spin through all
#ct = crime_type[3] # spin through all

In [ ]:
r = race[0] # spin through all
## Select the right filters
one_checkbox(r, "true", "Race", val=0) # toggle 
#time.sleep(5)
for ct in crime_type:
    one_checkbox(ct, "true", "Crime Type", val=0) # toggle
    ## Append the data to DF
    sentence_length_all = sentence_length_all.append(sentence_length(driver, r, ct), ignore_index=True)
    ## Reset the filters - Crime Type
    toggle_dropdown(driver, 'Crime Type')
    time.sleep(5)
    unselect_all(driver, crime_type)
    time.sleep(5)
## Reset the filters - Race
toggle_dropdown(driver, 'Race')
time.sleep(5)
unselect_all(driver, race)
time.sleep(5)

Element is a checkbox
Checkbox updated: White
Element is a checkbox
Checkbox updated: Administration of Justice
DF has been generated for race White, crime type Administration of Justice
All checkboxes toggled to off
Element is a checkbox
Checkbox updated: Antitrust
DF has been generated for race White, crime type Antitrust
All checkboxes toggled to off
Element is a checkbox
Checkbox updated: Arson


In [75]:
sentence_length_all

,Race,Crime Type,Distribution of Sentence Length 60 to 119 Months,Distribution of Sentence Length 24 to 59 Months,Distribution of Sentence Length Up to 24 Months,Distribution of Imprisonment Length 60 to 119 Months,Distribution of Imprisonment Length 24 to 59 Months,Distribution of Imprisonment Length Up to 24 Months,Sentence Length (Average Months),Sentence Length (Median Months),Imprisonment Length (Average Months),Imprisonment Length (Median Months),Supervised Release (Average Months),Supervised Release (Median Months),Distribution of Sentence Length 120 Months or More,Distribution of Imprisonment Length 120 Months or More
0,White,Administration of Justice,2.0%,11.7%,86.3%,3.1%,16.3%,80.6%,10,6,14,10,23,24,NaN,NaN
1,White,Antitrust,NaN,NaN,100.0%,NaN,NaN,100.0%,5,4,4,4,36,36,NaN,NaN
2,White,Assault,13.5%,19.0%,47.6%,17.5%,24.7%,32.0%,69,29,89,45,38,36,19.8%,25.8%


# Beautiful Soup - Sentencing outcomes

## Plea Status

In [ ]:
def all_plea_status(race):
    plea_status_all = pd.DataFrame()
    for i in race:
        tick_checkbox(i, "true", "Race")
        time.sleep(5)
        expand_list()
        plea_status_all = plea_status_all.append(plea_status(i), ignore_index=True)
        time.sleep(5)
        tick_checkbox(i, "None", "Race")
        time.sleep(5)
    return(plea_status_all)

In [ ]:
race = ['White','Black','Hispanic', 'Other']
plea_status_all = all_plea_status(race)
plea_status_all

In [ ]:
plea_status_all.to_csv('Plea Status All.csv')

## Sentence Type

In [ ]:
def all_sentencing_type(race):
    plea_status_all = pd.DataFrame()
    for i in race:
        tick_checkbox(i, "true", "Race")
        time.sleep(5)
        expand_list()
        time.sleep(5)
        sentencing_type_all = sentencing_type_all.append(sentence_type(i), ignore_index=True)
        time.sleep(5)
        tick_checkbox(i, "None", "Race")
        time.sleep(5)
    return(sentencing_type_all)


In [ ]:
race = ['White','Black','Hispanic', 'Other']
sentencing_type_all = all_sentencing_type(race)
sentencing_type_all

In [ ]:
sentence_type_all = sentence_type_all.append(sentence_type("Other"), ignore_index=True)

In [ ]:
sentence_type_all